In [1]:
%matplotlib inline
import os

import librosa
import librosa.display
import madmom
import matplotlib.pyplot as plt
import numpy as np
import numpy as np
import pyrubberband as pyrb
import soundfile
import soundfile as sf
import soundfile as sf
import torch
import torch
import torch.nn as nn
from IPython.display import Audio
from madmom.features.downbeats import DBNDownBeatTrackingProcessor, RNNDownBeatProcessor
from pydub import AudioSegment
from scipy.io.wavfile import write
from scipy.signal import find_peaks
from scipy.stats import mode
from sklearn.metrics import *
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset
from torch.utils.data import DataLoader, TensorDataset
from torch.utils.data import TensorDataset, DataLoader

ImportError: cannot import name 'MutableSequence' from 'collections' (C:\Users\sycom\miniconda3\envs\AI-DJ-Mix-Generator\Lib\collections\__init__.py)

In [ ]:
def detect_downbeats(audio_file, sr=44100, fps=100):
    # Load the audio signal without resampling
    signal = madmom.audio.signal.Signal(audio_file, sample_rate=sr, num_channels=1)
    
    # Get the beat and downbeat activations
    act = madmom.features.RNNDownBeatProcessor()(signal)
    
    # Create a downbeat processor
    proc = madmom.features.DBNDownBeatTrackingProcessor(beats_per_bar=[4], fps=fps)
    
    # Get the beats and downbeats
    beats_and_downbeats = proc(act)
    
    # Filter downbeats (those with beat position 1)
    downbeats = [beat for beat in beats_and_downbeats if beat[1] == 1]

    return np.array(downbeats)

from scipy.stats import mode

def estimate_tempo_from_downbeats(audio_file, downbeats):
    
    # Calculate the time difference between consecutive downbeats
    downbeat_differences = np.diff(downbeats[:, 0])

    # Calculate the time difference between consecutive downbeats
    downbeat_differences = np.around(np.diff(downbeats[:, 0]), decimals=6)

    # Get the mode of the differences
    mod_diff = mode(downbeat_differences).mode

    # Calculate the tempo: 60 seconds divided by the average difference
    # Since downbeat_differences are in seconds, this gives beats per minute
    tempo = 4 * (60 / mod_diff)
    
    tempo = round(tempo)

    return tempo, mod_diff, downbeat_differences

def estimate_beat(audio_file, sr=44100):
    try:
        # Load the audio signal with the specified sample rate
        signal = madmom.audio.signal.Signal(audio_file, sample_rate=sr, num_channels=1)
        
        # Consider only 4/4 time signature
        proc = DBNDownBeatTrackingProcessor(beats_per_bar=[4], fps=100)
        act = RNNDownBeatProcessor()(signal)
        proc_res = proc(act)
        
        # Extract the beat timestamps
        beat_timestamps = proc_res[:, 0]
        
        return beat_timestamps
    
    except Exception as e:
        print(e)
        return np.array([])  # Return an empty array for better consistency

In [ ]:
class SimpleTrack:
    def __init__(self, name, wav_file):
        self.name = name
        self.wav_file = wav_file
        self.audio, self.sr = librosa.load(wav_file, sr=None)
        self.downbeats = None
        self.beats = None
        self.tempo = None
        self.cueyes = None
        self.cueno = None
        self.time_signature = None
        self.x_tensor = None
        self.cue_points = None
        self.cue_points_indeces = None 
        
    def prepare_features_for_prediction(self):
        X_new = [self.convert_to_feature_vector(f) for f in self.features]
        X_new = scaler.transform(X_new)  # Normalize
        self.x_tensor = torch.FloatTensor(X_new).unsqueeze(0)
        
    
    def detect_downbeats(self):
        self.downbeats = detect_downbeats(self.wav_file)

    def estimate_tempo_from_downbeats(self):
        self.tempo, _, self.downbeat_differences = estimate_tempo_from_downbeats(self.wav_file, self.downbeats)
        
    def estimate_beat(self):
        self.beats = estimate_beat(self.wav_file)
        
    # 1. Convert the extracted features into flat feature vectors
    def convert_to_feature_vector(self, beat_features):
        """Convert beat features into a flattened feature vector."""
        flat_features = list(beat_features['MFCC']) + \
                        list(beat_features['Spectral Contrast']) + \
                        [beat_features['Spectral Centroid']] + \
                        [beat_features['Spectral Rolloff']] + \
                        [beat_features['Spectral Flux']] + \
                        [beat_features['RMS']]
        return flat_features

In [ ]:
# specify your path for the training set 

path = "kasi"

raw_wav_files = [os.path.join(path, file_name) for file_name in os.listdir(path) if file_name.endswith('.wav')]

raw_tracks = {}
counter = {}

for wav_file in raw_wav_files:
    track_name = (os.path.basename(wav_file))
    if track_name in counter:
        counter[track_name] += 1
        track_name = f"{track_name}{counter[track_name]}"
    else:
        counter[track_name] = 1
    track = SimpleTrack(track_name, wav_file)
    raw_tracks[track_name] = track

tracks = raw_tracks

In [ ]:
tracks

In [ ]:
def preprocess_simple_track(track):
    
    # Detect downbeats
    track.detect_downbeats()
    
    # Estimate tempo from downbeats
    track.estimate_tempo_from_downbeats()
    
    print(f"Estimated Tempo for {track.name}: {track.tempo:.2f} BPM")
    track.estimate_beat()

In [ ]:
# Calling the preprocess function on each track in the tracks dictionary
for track_name, track_obj in tracks.items():
    print(f"Preprocessing : {track_obj.name}")
    preprocess_simple_track(track_obj)

### REKORDBOX 

In [ ]:
aligned_track_names = {
    'Amniote Editions - JD Typo - Txtural': 'Amniote',
    'BEADS - SET THE TEMPO': 'BEADS',
    'DJ SWISHERMAN - Big Boy [RYS002]': 'DJ',
    'Gene Richards Jr - The Feeder (Chlär & Chontane Remix)': 'Gene',
    'JASØN - JASØN - Fura (Edvvin Remix)': 'JASØN',
    'kvlr - The Navigator [MASTER]': 'kvlr',
    'MiniTribe - Pharpheonix': 'MiniTribe',
    'PETERBLUE - Latin Club (Reformist Remix)': 'PETERBLUE',
    'Seigg - Needless': 'Seigg',
    "SJUSH - Pacman's Deathwish": 'SJUSH',
    'Shannen Blessing - Take Me Out': 'Shannen',
    'Stef de Haan - In 4 The Kill (Stef de Haan Remix)': 'Stef',
    'Bad Boombox - RAW Summer Hits 4 • Original Tracks - 20 Gushy (Ft. MC Yung Lil)': 'Bad',
    'CAIVA - RAW Summer Hits 4 • Original Tracks - 17 The Love Inside Me': 'CAIVA',
    'Dallaniel - RAW Summer Hits 4 • Original Tracks - 14 Somebody Else': 'Dallaniel',
    'Dylan Fogarty - RAW Summer Hits 4 • Original Tracks - 02 Da Funky Track': 'Dylan',
    'Mac Declos & LAZE - RAW Summer Hits 4 • Original Tracks - 19 Wanted Prisma': 'Mac',
    'RUIZ OSC1 - RAW Summer Hits 4 • Original Tracks - 03 Snare 3000': 'RUIZ',
    'The Chronics - RAW Summer Hits 4 • Original Tracks - 13 Luz': 'The'}

In [ ]:
import xml.etree.ElementTree as ET

def extract_hot_cues_for_aligned_tracks(xml_file_path, aligned_track_names):
    tree = ET.parse(xml_file_path)
    root = tree.getroot()

    hot_cue_points = {}
    
    # Iterate over all tracks in the XML
    for track in root.findall(".//TRACK"):
        track_name = track.get("Name")
        if track_name not in aligned_track_names:
            continue  # skip tracks not in the alignment
        
        cues = []
        for cue in track.findall(".//POSITION_MARK"):
            # Convert the start time from milliseconds to seconds
            start_time = float(cue.get("Start"))
            cues.append(start_time)
            
        hot_cue_points[track_name] = cues

    return hot_cue_points

# Usage:
rekordbox_xml_path = "rekordbox_meta_data.xml"
cue_points = extract_hot_cues_for_aligned_tracks(rekordbox_xml_path, aligned_track_names)
print(cue_points)


In [ ]:
for track_name, track_obj in tracks.items():
    track_name_without_extension = track_name.replace('.wav', '')
    if track_name_without_extension in cue_points:
        track_obj.cueyes = cue_points[track_name_without_extension]
    else:
        print(f"No hot cues found for track name: {track_name}")

In [ ]:
for track_name, track_obj in tracks.items():
    print(f"Track Name: {track_name}")
    print(f"Hot Cue Points: {track_obj.cueyes}")
    print("=====================================")


In [ ]:
def plot_audio_segment_with_beats_and_cues(audio, sr, beats, cue_points, start_time, duration):
    # Calculate start and end samples
    start_sample = int(start_time * sr)
    end_sample = start_sample + int(duration * sr)

    # Create time array
    time = np.arange(start_sample, end_sample) / sr

    # Identify beats and cue points in the segment
    segment_beats = [beat for beat in beats if start_time <= beat < start_time + duration]

    segment_cues = [float(cue) for cue in cue_points if start_time <= float(cue) < start_time + duration]


    # Plot audio segment
    plt.figure(figsize=(10, 4))
    plt.plot(time, audio[start_sample:end_sample])
    plt.vlines(segment_beats, ymin=np.min(audio), ymax=np.max(audio), colors='r', label='Beats')
    plt.vlines(segment_cues, ymin=np.min(audio), ymax=np.max(audio), colors='b', linestyle='dashed', label='Cue Points')
    plt.xlabel('Time (s)')
    plt.ylabel('Amplitude')
    plt.title('Audio Segment with Beats and Cue Points')
    plt.legend()
    plt.xlim([start_time, start_time + duration])
    plt.show()
    


In [ ]:
def plot_audio_segment_around_first_and_last_cue(audio, sr, beats, cue_points):
    # Extract only the first and last cue points
    first_cue = cue_points[1]
    last_cue = cue_points[-2]
    selected_cue_points = [first_cue, last_cue]
    
    for cue in selected_cue_points:
        cue = float(cue)  # Convert cue point from string to float
        
        # Define start time and duration for the segment centered around the cue point
        start_time = max(cue - 10, 0)  # Ensure the start time is not negative
        duration = 20
        
        # Calculate end time, ensuring it doesn't exceed the length of the audio
        end_time = min(cue + 10, len(audio) / sr)
        
        # Adjust duration if end_time is at the end of the audio
        if end_time == len(audio) / sr:
            start_time = end_time - 20
        
        # If cue point is near the beginning or end, adjust start_time to ensure cue is within plotting window
        if cue - start_time < 5:  # If cue is within first 5 seconds of window
            start_time = cue - 5
        elif end_time - cue < 5:  # If cue is within last 5 seconds of window
            start_time = cue - 15

        # Ensure start_time and duration values are valid
        start_time = max(0, start_time)
        duration = min(duration, len(audio) / sr - start_time)

        # Plot the segment
        plot_audio_segment_with_beats_and_cues(audio, sr, beats, np.array(selected_cue_points), start_time, duration)

for track_name, track in tracks.items():
    print(f"Plotting for track: {track_name}")
    audio = track.audio
    sr = track.sr
    beats = track.beats
    cue_points = track.cueyes  # Use the cue_points attribute of the track
    plot_audio_segment_around_first_and_last_cue(audio, sr, beats, cue_points)

In [ ]:
def label_cues(beats, cueyes):
    """Label the cues and their neighbors."""
    # Start with an array of -1 (unlabeled)
    labels = np.ones(len(beats)) * -1

    # Convert cueyes to indices
    cueyes_indices = []
    for cue in cueyes:
        cue_index = np.argmin(np.abs(np.array(beats) - cue))
        cueyes_indices.append(cue_index)
        labels[cue_index] = 1

    # Label the neighboring beats within the radius
    cueno_indices = []
    for idx in cueyes_indices:
        for offset in range(-3, 4):  # Include the range from -3 to 3
            if 0 <= idx + offset < len(beats) and offset != 0:  # Check boundaries and exclude the cue itself
                labels[idx + offset] = 0
                cueno_indices.append(idx + offset)

    # Diagnostic print statements
    print("Cueyes Indices:", cueyes_indices)
    print("Cueno Indices:", cueno_indices)

    return labels, cueyes_indices, cueno_indices

# Apply the function to all tracks
for track_name, track in tracks.items():
    track.labels, track.cueyes, track.cueno = label_cues(track.beats, track.cueyes)

In [ ]:
total_cueyes = 0
total_cueno = 0

for track_name, track in tracks.items():
    total_cueyes += len(track.cueyes)
    total_cueno += len(track.cueno)

print(f"Total Cueyes: {total_cueyes}")
print(f"Total Cueno: {total_cueno}")

In [ ]:
import librosa
import numpy as np

def extract_features(audio, sr, timestamp, window_size=1.0):
    """
    Extract multiple audio features for a given timestamp using librosa.
    
    Parameters:
    - audio: The audio data as a numpy array.
    - sr: Sample rate.
    - timestamp: The given timestamp (in seconds).
    - window_size: The size of the window around the timestamp (in seconds, default is 1 second).
    
    Returns:
    A dictionary with feature names as keys and extracted values as values.
    """

    # Define the start and end samples for the window
    center_sample = int(timestamp * sr)
    start_sample = center_sample - int(sr * window_size / 2)
    end_sample = center_sample + int(sr * window_size / 2)
    
    # Check for edge cases
    if start_sample < 0:
        start_sample = 0
    if end_sample > len(audio):
        end_sample = len(audio)

    windowed_audio = audio[start_sample:end_sample]

    # If windowed_audio is too small, return an empty dictionary
    if len(windowed_audio) <= 1:
        return {}

    # Ensure even size for windowed_audio by padding with zero if necessary
    if len(windowed_audio) % 2 != 0:
        windowed_audio = np.append(windowed_audio, 0)

    # Extract MFCC
    mfcc_coeffs = librosa.feature.mfcc(y=windowed_audio, sr=sr, n_mfcc=13)

    # Extract Spectral Centroid
    spec_centroid = librosa.feature.spectral_centroid(y=windowed_audio, sr=sr)

    # Extract Spectral Contrast
    spec_contrast = librosa.feature.spectral_contrast(y=windowed_audio, sr=sr)

    # Extract Spectral Rolloff
    spec_rolloff = librosa.feature.spectral_rolloff(y=windowed_audio, sr=sr)

    # Extract Spectral Flux - this is the difference between consecutive spectral frames
    spectrum = librosa.stft(windowed_audio)
    flux = np.diff(np.abs(spectrum), axis=1)

    # Calculate RMS for the beat
    rms = librosa.feature.rms(y=windowed_audio)

    features = {
        "MFCC": mfcc_coeffs.mean(axis=1),  # Averaging over time frames
        "Spectral Centroid": np.mean(spec_centroid),
        "Spectral Contrast": np.mean(spec_contrast, axis=1),  # Averaging over frequency bands
        "Spectral Rolloff": np.mean(spec_rolloff),
        "Spectral Flux": np.mean(flux),
        "RMS": np.mean(rms)
    }

    return features

In [ ]:
def extract_features_for_labeled_beats(track):
    all_features = []
    for idx, beat_time in enumerate(track.beats):
        # Check if the beat has a label of 1 (cueyes) or 0 (cueno)
        if track.labels[idx] in [0, 1]:
            features = extract_features(track.audio, track.sr, timestamp=beat_time)
            features['label'] = 1 if track.labels[idx] == 1 else 0  # 'CUE' or 'NON-CUE' can be used instead of 1 or 0 if needed
            all_features.append(features)
    return all_features

# Extract features for all tracks and check the lengths
for track_name, track in tracks.items():
    track.features = extract_features_for_labeled_beats(track)
    print(f"Track: {track_name}")
    print(f"Expected Features (from cueyes + cueno): {len(track.cueyes) + len(track.cueno)}")
    print(f"Actual Extracted Features: {len(track.features)}")
    print("------")

In [ ]:
def convert_to_feature_vector(beat_features):
    """Convert beat features into a flattened feature vector."""
    # Flatten MFCC and Spectral Contrast
    flat_features = list(beat_features['MFCC']) + \
                    list(beat_features['Spectral Contrast']) + \
                    [beat_features['Spectral Centroid']] + \
                    [beat_features['Spectral Rolloff']] + \
                    [beat_features['Spectral Flux']] + \
                    [beat_features['RMS']]
    return flat_features

X = []
y = []

for track_name, track in tracks.items():
    for features in track.features:
        # Extract features and labels
        X.append(convert_to_feature_vector(features))
        y.append(features['label'])

X = np.array(X)
y = np.array(y)


In [ ]:
# Split the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Normalize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

import torch

# Convert data to PyTorch tensors
X_train_tensor = torch.FloatTensor(X_train)
y_train_tensor = torch.FloatTensor(y_train)
X_test_tensor = torch.FloatTensor(X_test)
y_test_tensor = torch.FloatTensor(y_test)


# Reshaping the data to add an additional sequence dimension
X_train_tensor = X_train_tensor.unsqueeze(1)  # shape becomes [batch_size, 1, input_dim]
X_test_tensor = X_test_tensor.unsqueeze(1)    # shape becomes [batch_size, 1, input_dim]


def init_hidden(batch_size):
    return (torch.zeros(num_layers, batch_size, hidden_dim),
            torch.zeros(num_layers, batch_size, hidden_dim))

# Define the LSTM model without dropout
class LSTMNet(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers):
        super(LSTMNet, self).__init__()
        
        # LSTM layer
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        
        # Fully connected layer
        self.fc = nn.Linear(hidden_dim, output_dim)
    
    def forward(self, x, hidden):
        # LSTM layer
        out, hidden = self.lstm(x, hidden)
    
        # Get output for each time step in the sequence
        out = self.fc(out).squeeze(1)  # We squeeze the sequence length dimension, which is 1 in this case
    
        return torch.sigmoid(out), hidden
    
hidden_dim = 10
num_layers = 1
input_dim = X_train.shape[1]
output_dim = 1
    
lstm_model = LSTMNet(input_dim, hidden_dim, output_dim, num_layers)

# Xavier initialization
for name, param in lstm_model.named_parameters():
    if 'bias' in name:
        nn.init.constant_(param, 0.0)
    elif 'weight' in name:
        nn.init.xavier_normal_(param)


# Define the loss function and optimizer with reduced learning rate
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(lstm_model.parameters(), lr=0.0005)  

# Training loop
num_epochs = 100
train_losses = []
batch_size = 8  # As discussed

# Training loop
train_losses = []

for epoch in range(num_epochs):
    for i in range(0, len(X_train_tensor) - batch_size + 1, batch_size):  
        
        # Get mini-batch
        inputs = X_train_tensor[i:i+batch_size]
        labels = y_train_tensor[i:i+batch_size]

        # Ensure the batch size is consistent for inputs and labels
        actual_batch_size = min(batch_size, len(inputs), len(labels))
        inputs = inputs[:actual_batch_size]
        labels = labels[:actual_batch_size]

        # Initialize hidden states
        h0, c0 = init_hidden(actual_batch_size)
        
        # Zero the gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs, _ = lstm_model(inputs, (h0, c0))
        loss = criterion(outputs.squeeze(), labels)
        
        # Backward pass and optimization
        loss.backward()
        
        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(lstm_model.parameters(), max_norm=1)
        optimizer.step()
    
    train_losses.append(loss.item())

    # Print loss every 10 epochs
    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

final_train_loss = train_losses[-1]  # Return the final training loss
# final_train_loss

# 4. Evaluation 

lstm_model.eval()
with torch.no_grad():
    h0, c0 = init_hidden(len(X_test_tensor))
    outputs, _ = lstm_model(X_test_tensor, (h0, c0))
    predictions = (outputs.squeeze() > 0.5).float()

predictions = predictions.numpy()
y_test_flattened = y_test_tensor.numpy()

accuracy = accuracy_score(y_test_flattened, predictions)
precision = precision_score(y_test_flattened, predictions)


In [ ]:
print("Results: ")
print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")

### Saving the model 

In [ ]:
# it was lstm model 1 but dont want to override it 

torch.save(lstm_model.state_dict(), "lstm_model_demo.pth")

# same here for the scaler 

from joblib import dump

dump(scaler, 'scaler_demo.joblib')

### Testing the Cue Point Generator Model on unseen data
### 1. Alec Dienar - Perpetual Groove EP 
### 2. Impulsive Behaviour - unreleased tracks 

In [ ]:
# Specify your paths
path1 = "AlecDienaar-PerpetualGroove"
path2 = "ImpulsiveBehaviour-Unreleased"

paths = [path1, path2]
all_tracks = {}
counter = {}

for path in paths:
    wav_files = [os.path.join(path, file_name) for file_name in os.listdir(path) if file_name.endswith(('.wav', '.aif'))]

    for wav_file in wav_files:
        track_name = (os.path.basename(wav_file))
        if track_name in counter:
            counter[track_name] += 1
            track_name = f"{track_name}{counter[track_name]}"
        else:
            counter[track_name] = 1
        track = SimpleTrack(track_name, wav_file)
        all_tracks[track_name] = track


In [ ]:
all_tracks

In [ ]:
# Calling the preprocess function on each track in the all_tracks dictionary
for track_name, track_obj in all_tracks.items():
    print(f"Preprocessing : {track_obj.name}")
    preprocess_simple_track(track_obj)

In [ ]:
new_data_dict = all_tracks

In [ ]:
def extract_features_for_all_beats(track):
    all_features = []
    for beat_time in track.beats:
        features = extract_features(track.audio, track.sr, timestamp=beat_time)
        # If you still want to include the labels for some tracks (like the training tracks), you can do:
        # features['label'] = 1 if track.labels[idx] == 1 else 0
        all_features.append(features)
    return all_features

# Extract features for all tracks and check the lengths
for track_name, track in new_data_dict.items():
    track.features = extract_features_for_all_beats(track)

In [ ]:
from bisect import bisect_left

def get_cue_point_timestamps(track, cue_point_indices):
    return [track.beats[i] for i in cue_point_indices]

def init_hidden(batch_size):
    return (torch.zeros(num_layers, batch_size, hidden_dim),
            torch.zeros(num_layers, batch_size, hidden_dim))

class LSTMNet(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers):
        super(LSTMNet, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
    
    def forward(self, x, hidden):
        out, hidden = self.lstm(x, hidden)
        out = self.fc(out).squeeze(1)
        return torch.sigmoid(out), hidden
    
# Parameters 

hidden_dim = 10
num_layers = 1
input_dim = 24 # length of the flat feature vector
output_dim = 1

from joblib import load

# Load the scaler and model
scaler = load('scaler_demo.joblib')
model = LSTMNet(input_dim, hidden_dim, output_dim, num_layers)
model.load_state_dict(torch.load("lstm_model_demo.pth"))
model.eval()  # Set the model to evaluation mode
    
def categorize_cue_points(cue_points):
    # Define the four arrays
    arrays = {
        0: [],
        1: [],
        2: [],
        3: []
    }
    
    for point in cue_points:
        remainder = point % 4
        arrays[remainder].append(point)
    
    # Find the array with the most cue points
    max_key = max(arrays, key=lambda k: len(arrays[k]))
    
    return max_key
    
def detect_cue_points_for_track(track, num_cue_points=12, filter_by=None):
    with torch.no_grad():
        h0, c0 = init_hidden(1)  # Batch size of 1
        outputs, _ = model(track.x_tensor, (h0, c0))
        predicted_cues = outputs.squeeze().tolist()

    # If filtering is applied
    if filter_by is not None:
        filtered_indices = [i for i in range(len(predicted_cues)) if i % 4 == filter_by]
        filtered_cues = [predicted_cues[i] for i in filtered_indices]
    else:
        filtered_cues = predicted_cues
        filtered_indices = list(range(len(predicted_cues)))

    # Split the track into n intervals and get the top cue point in each interval
    interval_length = len(predicted_cues) // num_cue_points  # We use the full track's length
    cue_indices = []

    for i in range(num_cue_points):
        start = i * interval_length
        end = (i + 1) * interval_length
        
        # Get the corresponding start and end indices for the filtered cues
        filtered_start = bisect_left(filtered_indices, start)
        filtered_end = bisect_left(filtered_indices, end)
        
        # Clip the end if it exceeds the filtered cues length
        filtered_end = min(filtered_end, len(filtered_cues) - 1)
        
        interval_values = filtered_cues[filtered_start:filtered_end]
        
        # Skip if interval_values is empty
        if not interval_values:
            continue
        
        max_index_in_interval = np.argmax(interval_values)
        original_index = filtered_indices[filtered_start + max_index_in_interval]
        cue_indices.append(original_index)

    track.cue_points_indeces = np.array(cue_indices)
    track.cue_points = get_cue_point_timestamps(track, track.cue_points_indeces)
    


In [ ]:
for track_name, track in new_data_dict.items():
    track.prepare_features_for_prediction()
    
    # First run to determine the dominant array
    detect_cue_points_for_track(track, num_cue_points=64)
    dominant_key = categorize_cue_points(track.cue_points_indeces)
    
    # Second run with filtering
    detect_cue_points_for_track(track, num_cue_points=12, filter_by=dominant_key)
    
    print(f"Track: {track_name}")
    print(f"Cue Points (indices): {track.cue_points_indeces}")
    print(f"Cue Points: {track.cue_points}")
    print("------")

In [ ]:
def plot_waveform_with_hot_cues(audio, name, sr, hot_cues):
    # Create a time array for the waveform
    duration = len(audio) / sr
    times = np.linspace(0, duration, len(audio))

    # Create the plot
    plt.figure(figsize=(14, 5))
    plt.plot(times, audio, alpha=0.6)

    # Plot the hot cues as vertical lines
    for hot_cue in hot_cues:
        plt.axvline(x=hot_cue, color='r')

    # Get the first word of the name
    first_word = name.split(' ')[0]

    plt.ylim(-1, 1)
    plt.xlabel('Time (s)')
    plt.ylabel('Amplitude')
    plt.title(f"Waveform with Hot Cues: {first_word}")
    plt.show()

In [ ]:
new_data_dict

In [ ]:
for track_name, track in new_data_dict.items():
    plot_waveform_with_hot_cues(track.audio, track.name, track.sr, track.cue_points)